##### Parameters
- workspaceName - The name of the workspace where the source warehouse exists.
- warehouseName - The name of the warehouse that is to be backed up.
- workspaceBackupName - The name of the workspace that the lakehouse for backups exists. This defaults to the workspace where this notebook exists.
- lakehouseBackupName - The name of the lakehouse used for back ups.

In [ ]:
workspaceName = 'WS_Demo_InternetSales'
warehouseName = 'WH_InternetSales'

In [ ]:
workspaceBackupName = spark.conf.get('trident.workspace.id')
lakehouseBackupsName = 'LH_DW_Backups'

##### Copy the data from the Warehouse delta folders a separate Lakehouse

In [ ]:
import datetime
from notebookutils import mssparkutils

def deep_ls(path: str, max_depth=1):
    """
    https://www.rakirahman.me/directory-recursion-synapse/

    List all files and folders in specified path and
    subfolders within maximum recursion depth.
    """

    # List all files in path
    li = mssparkutils.fs.ls(path)

    # Return all files
    for x in li:
        if x.size != 0:
            yield x

    # If the max_depth has not been reached, start
    # listing files and folders in subdirectories
    if max_depth > 1:
        for x in li:
            if x.size != 0:
                continue
            for y in deep_ls(x.path, max_depth - 1):
                yield y

    # If max_depth has been reached,
    # return the folders
    else:
        for x in li:
            if x.size == 0:
                yield x


fileList = deep_ls(f'abfss://{workspaceName}@onelake.dfs.fabric.microsoft.com/{warehouseName}.datawarehouse/Tables', max_depth=2)

nowDatetime = datetime.datetime.now(datetime.timezone.utc)
backupDatetime = nowDatetime.strftime("%Y%m%d_%H%M")
print(f'{workspaceName = }')
print(f'{backupDatetime = }')

# https://blog.fabric.microsoft.com/en-us/blog/optimizing-spark-compute-for-medallion-architectures-in-microsoft-fabric?ft=Santhosh%20Kumar%20Ravindran:author
spark.conf.set("spark.sql.parquet.vorder.enabled","false")
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled","false")
spark.conf.set("spark.databricks.delta.collect.stats","false")

for file in fileList:
    schema = file.path.split('/')[-2]
    table = file.name
    print(f'{schema = }, {table = }')
    df = spark.read.format('delta').load(file.path)
    df.write.mode('overwrite').format('delta').save(f'abfss://{workspaceBackupName}@onelake.dfs.fabric.microsoft.com/{lakehouseBackupsName}.Lakehouse/Files/{workspaceName}/{warehouseName}/{backupDatetime}/{schema}/{table}')


##### Drop old backups - backupRetentionCnt is set to 5 by default meaning it will keep the last 5 backups.

In [ ]:
backupList = mssparkutils.fs.ls(f'abfss://{workspaceBackupName}@onelake.dfs.fabric.microsoft.com/{lakehouseBackupsName}.Lakehouse/Files/{workspaceName}/{warehouseName}')
backupRetentionCnt = 5

for i, file in enumerate(backupList):
    if len(backupList) - i > backupRetentionCnt:
        print(f'Removing directory "{file.path}"')
        mssparkutils.fs.rm(file.path, recurse=True)
    else:
        break